Import the modules, create a database connection.

In [2]:
import psycopg2
from pgvector.psycopg2 import register_vector
import json
import skimage
import xml.etree.ElementTree as ET

## Setup the database connection

Set the host to your Azure Database for PostgreSQL instance, be sure to replace the `SUFFIX`.

In [3]:
#connect to the database
username = 'wsuser'
password = 'Solliance123'
host = 'pgsqldevSUFFIXflex16.postgres.database.azure.com'
port = '5432'
dbname = 'ailabs'

dbconn = psycopg2.connect(host=host, user=username, password=password, port=port, database=dbname , connect_timeout=10)

dbconn.set_session(autocommit=True)

Enable the vector extension and call register_verctor on the database connection.

In [4]:
#This will take ~8-10mins to execute...
cur = dbconn.cursor()
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")

register_vector(dbconn)

Create the database tables.

In [15]:
#create the tables

cur = dbconn.cursor()
cur.execute('DROP TABLE IF EXISTS recipes')
cur.execute('DROP TABLE IF EXISTS ingredients')
cur.execute('DROP TABLE IF EXISTS recipe_ingredients')

cur.execute("""CREATE TABLE public.ingredients (
    id integer NOT NULL,
    name text
);
""")

cur.execute("""CREATE TABLE public.recipe_ingredients (
    id integer NOT NULL,
    recipe_id integer,
    ingredient_id integer,
    quantity text,
    unit_key integer
);
""")

cur.execute("""
        CREATE TABLE public.recipes (
        id integer NOT NULL,
        name text,
        description text,
        category text,
        embedding public.vector(1536),
        compacted_embedding public.vector(50),
        super_compacted_embedding public.vector(10)
    );
""")

Create the import and export functions.

In [20]:
import pandas as pd
#load the table into dataframe
def export_data(engine):
    df = pd.read_sql_query('SELECT * FROM recipes',con=engine)
    #output to csv
    df.to_csv('./data/recipes.csv', index=False)

    df = pd.read_sql_query('SELECT * FROM recipe_ingredients',con=engine)
    #output to csv
    df.to_csv('./data/recipe_ingredients.csv', index=False)

    df = pd.read_sql_query('SELECT * FROM ingredients',con=engine)
    #output to csv
    df.to_csv('./data/ingredients.csv', index=False)

def import_data(engine):

    df = pd.read_csv('./data/recipe_ingredients.csv')
    df.to_sql('recipe_ingredients', con=engine, if_exists='append', index=False)

    df = pd.read_csv('./data/ingredients.csv')
    df.to_sql('ingredients', con=engine, if_exists='append', index=False)

    df = pd.read_csv('./data/recipes.csv')
    df.to_sql('recipes', con=engine, if_exists='append', index=False)

Create a SQL Alchemy engine for the import/export.

In [21]:
from sqlalchemy import create_engine
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{dbname}')
import_data(engine)

Perform a search for similar items as `dish, turkey, curry`.

In [22]:
#find similar items
sql = """
SELECT
        recipe_1.id,
        recipe_1.name,
        recipe_2.id,
        recipe_2.name
FROM (SELECT * FROM recipes WHERE name = 'Dish, turkey, curry') recipe_1,
        recipes AS recipe_2
WHERE recipe_1.id != recipe_2.id
ORDER BY recipe_1.embedding <=> recipe_2.embedding
LIMIT 10;
"""

#execute the query
cur = dbconn.cursor()
cur.execute(sql)
results = cur.fetchall()
for row in results:
    print(row)

(272, 'Dish, turkey, curry', 271, 'Dish, turkey & noodles, baked')
(272, 'Dish, turkey, curry', 251, 'Dish, pot pie, turkey')
(272, 'Dish, turkey, curry', 675, 'Soup, tomato')
(272, 'Dish, turkey, curry', 659, 'Soup, chicken rice')
(272, 'Dish, turkey, curry', 672, 'Soup, rice w/beef')
(272, 'Dish, turkey, curry', 660, 'Soup, chicken vegetable/Mulligatawny')
(272, 'Dish, turkey, curry', 199, 'Dish, chicken, a la king')
(272, 'Dish, turkey, curry', 38, 'Beef, simmered')
(272, 'Dish, turkey, curry', 689, 'Stuffing, savory bread')
(272, 'Dish, turkey, curry', 686, 'Stew, beef chunks, w/juices & veg, cnd')


Perform a search for items that do not seem to be similar to a `corn dog`.

In [23]:
#find items that are not similar
sql = """
SELECT
        recipe_1.id,
        recipe_1.name,
        recipe_2.id,
        recipe_2.name
FROM (SELECT * FROM recipes WHERE name = 'Corn Dog' LIMIT 1) recipe_1,
        recipes AS recipe_2
WHERE
  recipe_2.description IS NOT NULL
ORDER BY recipe_1.embedding <=> recipe_2.embedding DESC
LIMIT 2;
"""

#execute the query
cur = dbconn.cursor()
cur.execute(sql)
results = cur.fetchall()
for row in results:
    print(row)

(165, 'Corn Dog', 538, 'Salad, green, tossed')
(165, 'Corn Dog', 519, 'Salad, chicken, tropical')
